In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import gkey

In [2]:
# read weather data into a dataframe

weather_df = pd.read_csv('../output_data/cities.csv')
weather_df.head()

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Date
0,Kapaa,US,22.0752,-159.3190,78.80,61,12.66,40,1618270419
1,Arraial do Cabo,BR,-22.9661,-42.0278,77.32,86,19.66,1,1618270601
2,Dzaoudzi,YT,-12.7887,45.2699,82.40,83,8.05,40,1618270601
3,Omsukchan,RU,62.5333,155.8000,-1.91,84,4.47,68,1618270601
4,Ribeira Grande,PT,38.5167,-28.7000,61.99,87,19.93,89,1618270602


In [3]:
# configure gmaps with API key

gmaps.configure(api_key = gkey)

City                object
Country             object
Latitude           float64
Longitude          float64
Max Temperature    float64
Humidity             int64
Wind Speed         float64
Cloudiness           int64
Date                 int64
dtype: object

In [4]:
# store latitude and longitude as location

locations = weather_df[['Latitude', 'Longitude']]


In [38]:
# create a humidity heatmap layer



fig = gmaps.figure(center = (25,0), zoom_level = 1.65)

heat_layer = gmaps.heatmap_layer(locations, weights = weather_df['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [39]:
# new data frame with weather criteria

low_temp = 50
high_temp = 75
hum_lvl = 70
cloud = 75

perfect_weather = weather_df.loc[(weather_df['Max Temperature'] >= low_temp) & 
                                 (weather_df['Max Temperature'] <= high_temp) &
                                 (weather_df['Humidity'] >= hum_lvl) & 
                                 (weather_df['Cloudiness'] >= cloud)
                                ]

In [40]:
# limit options for sanity

vacation_spots = perfect_weather.sample(n=5)
vacation_spots['Hotel'] = ''

In [41]:
# find a hotel within 5000 metres of city coordinates
# add hotel to dataframe

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000
marker_locations = []

for index, row in vacation_spots.iterrows():
    lat = vacation_spots.loc[index, 'Latitude']
    lng = vacation_spots.loc[index, 'Longitude']
    params = {
        'location' : f'{lat}, {lng}',
        'type' : 'hotel',
        'radius' : radius,
        'key' : gkey,
    }
    response = requests.get(base_url, params = params).json()
    vacation_spots.loc[index, 'Hotel'] = response['results'][1]['name']
    marker_locations.append((lat, lng))
    
vacation_spots

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Date,Hotel
183,Pontes e Lacerda,BR,-15.2261,-59.3353,70.39,95,1.32,77,1618270654,Hotel Bressan
559,Nosy Varika,MG,-20.5833,48.5333,72.66,93,6.58,98,1618270775,Nosy Varika Hospital
188,Shingū,JP,33.7333,135.9833,59.00,88,8.05,75,1618270656,Mehariya Shingu Honten
130,Esperance,AU,-33.8667,121.9000,51.80,100,18.41,75,1618270616,Hospitality Esperance
222,Wiarton,CA,44.7407,-81.1332,50.00,87,6.91,75,1618270667,Waterview On The Bay


In [42]:
# add hotel marks to cities

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig
    

Figure(layout=FigureLayout(height='420px'))